<a href="https://colab.research.google.com/github/duylinh55555/SpeechRecognition_Arduino33/blob/main/SpeechRecognition_MCU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cài đặt các gói liên quan

In [ ]:
! pip install tflite-model-maker

     |████████████████████████████████| 616 kB 13.7 MB/s 
     |████████████████████████████████| 87 kB 7.5 MB/s 
     |████████████████████████████████| 120 kB 44.7 MB/s 
     |████████████████████████████████| 840 kB 26.5 MB/s 
     |████████████████████████████████| 3.4 MB 49.4 MB/s 
     |████████████████████████████████| 1.2 MB 50.3 MB/s 
     |████████████████████████████████| 6.4 MB 51.8 MB/s 
     |████████████████████████████████| 1.1 MB 73.5 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 1.1 MB 55.0 MB/s 
     |████████████████████████████████| 596 kB 39.7 MB/s 
     |████████████████████████████████| 213 kB 64.9 MB/s 
     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 47.7 MB 94 kB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
     |████████████████████████████████| 352 kB 58.7 MB/s 
     |████████████████████████████████| 210 kB 55.4 MB/s 
  Created wheel for f

# Import thư viện

In [ ]:
import tensorflow as tf
import tflite_model_maker as maker
from tflite_model_maker import audio_classifier
import os

# Upload dataset

In [ ]:
birds_dataset_folder = tf.keras.utils.get_file('birds_dataset.zip',
                                                'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/birds_dataset.zip',
                                                cache_dir='./',
                                                cache_subdir='dataset',
                                                extract=True)

343695360/343680986 [==============================] - 7s 0us/step


hoặc ...

In [ ]:
from google.colab import files

uploaded = files.upload()

! gdown --id # my Google Drive file ID

# Chuẩn bị dữ liệu

In [ ]:
data_dir = './dataset/small_birds_dataset'

spec = audio_classifier.YamNetSpec(
    keep_yamnet_and_custom_heads=True,
    frame_step=3 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,
    frame_length=6 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH)

train_data = audio_classifier.DataLoader.from_folder(spec, os.path.join(data_dir, 'train'), cache=True)
train_data, validation_data = train_data.split(0.8)
test_data = audio_classifier.DataLoader.from_folder(spec, os.path.join(data_dir, 'test'), cache=True)

INFO:tensorflow:Checkpoints are stored in /tmp/tmppk9sr6yh


# Training & Evaluating

In [ ]:
batch_size = 128
epochs = 50

print('training time')
model = audio_classifier.create(train_data,spec,validation_data,batch_size=batch_size,epochs=epochs)

print('evaluation time')
model.evaluate(test_data)

models_path = './models'
model.export(models_path, export_format=[maker.ExportFormat.SAVED_MODEL, maker.ExportFormat.LABEL])

training time
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 classification_head (Dense)  (None, 5)                5125      
                                                                 
Total params: 5,125
Trainable params: 5,125
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
22/22 [==============================] - 261s 11s/step - loss: 1.4262 - acc: 0.4246 - val_loss: 1.4924 - val_acc: 0.2841
Epoch 2/50
22/22 [==============================] - 1s 32ms/step - loss: 1.1725 - acc: 0.5750 - val_loss: 1.3142 - val_acc: 0.4220
Epoch 3/50
22/22 [==============================] - 1s 32ms/step - loss: 1.0614 - acc: 0.6236 - val_loss: 1.2546 - val_acc: 0.4499
Epoch 4/50
22/22 [==============================] - 1s 32ms/step - loss: 0.9482 - acc: 0.6917 - val_loss: 1.1882 - val_acc: 0.5209
Epoch 5/50
22/22 [=======================

INFO:tensorflow:Assets written to: ./models/saved_model/assets


INFO:tensorflow:Assets written to: ./models/saved_model/assets


INFO:tensorflow:Saving labels in ./models/labels.txt


INFO:tensorflow:Saving labels in ./models/labels.txt


# Chuyển đổi model 

Quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('models/saved_model')

def representative_dataset_gen():
    for sample in x_test:
        yield [sample]
  
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

converter.representative_dataset = representative_dataset_gen
model_tflite = converter.convert()

open(MODEL_TFLITE, "wb").write(model_tflite) # model

NameError: ignored

Chuyển đổi về định dạng dùng cho MicroController

In [ ]:
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}

REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

# Xuất model

In [ ]:
models_path = './models'
model.export(models_path, tflite_filename='commands_model.tflite')

# Triển khai trên Arduino Nano 33 BLE

Triển khai trên Arduino Nano 33 BLE